# Reinforcement Learning for Trading

Author: Elkin Narváez Paz

* Email: elkinnarvaez2406@gmail.com
* LinkedIn: https://www.linkedin.com/in/elkinnarvaez/
* GitHub: https://github.com/elkinnarvaez

© All rights reserved.

## Libraries

In [19]:
# Data gathering
import http.client
import requests
import json
import datetime

# Statistics
import statistics as stats

## Data gathering

In [20]:
stockSymbols = ['MSFT', 'AAPL', 'AMZN', 'TSLA', 'GOOGL', 'GOOG', 'FB', 'NVDA', 'BRK-B', 'JPM', 'HD', 'JNJ', 'UNH', 'PG', 'BAC', 'V', 'ADBE', 'NFLX', 'CRM', 'PFE', 'DIS', 'MA', 'XOM', 'TMO', 'COST']
indicators = {}

conn = http.client.HTTPSConnection("yh-finance.p.rapidapi.com")

headers = {
    'x-rapidapi-host': "yh-finance.p.rapidapi.com",
    'x-rapidapi-key': "fc1708951fmshca8b1e8a9e96604p126192jsnf88ca47a0efd"
    }

In [21]:
for symbol in stockSymbols:
    print(symbol)
    conn.request("GET", "/stock/v2/get-chart?interval=60m&symbol=%s&range=2y&region=US"%(symbol), headers=headers)

    res = conn.getresponse()
    data = res.read()

    result = json.loads(data.decode("utf-8"))["chart"]["result"][0]

    indicators[symbol] = {
        "timestamp": list(map(lambda x: datetime.datetime.fromtimestamp(x), result["timestamp"])),
        "high": result["indicators"]["quote"][0]["high"],
        "volume": result["indicators"]["quote"][0]["volume"],
        "open": result["indicators"]["quote"][0]["open"],
        "low": result["indicators"]["quote"][0]["low"],
        "close": result["indicators"]["quote"][0]["close"]
    }

MSFT
AAPL
AMZN
TSLA
GOOGL
GOOG
FB
NVDA
BRK-B
JPM
HD
JNJ
UNH
PG
BAC
V
ADBE
NFLX
CRM
PFE
DIS
MA
XOM
TMO
COST


## Creation of trading signals

In [23]:
tradingSignals = {}
for symbol in stockSymbols:
    tradingSignals[symbol] = {}

### Simple Moving Average (SMA)

In [32]:
def SMA(symbol, numPeriods):
    """
        Input:
            - symbol: the symbol of the stock for which the SMA will be calculated to -> str
            - numPeriods: number of time periods over which to average -> int
        Output:
            SMA values
    """
    smaValues = []
    
    history = [] # history of prices
    for closePrice in indicators[symbol]['close']:
        history.append(closePrice)
        if(len(history) > numPeriods): # remove the oldest price because we only average over the last 'numPeriods' prices
            del history[0]
        smaValues.append(stats.mean(history))
        
    return smaValues

### Exponential Moving Average (EMA)

In [33]:
def EMA(symbol, numPeriods):
    """
        Input:
            - symbol: the symbol of the stock for which the EMA will be calculated to -> str
            - numPeriods: number of time periods over which to average -> int
        Output:
            EMA values
    """
    emaValues = []
    
    K = 2 / (numPeriods + 1) # smoothing constant
    ema = 0
    
    for closePrice in indicators[symbol]['close']:
        if(ema == 0):
            ema = closePrice
        else:
            ema = (closePrice - ema) * K + ema
        emaValues.append(ema)
    
    return emaValues

### Absolute Price Oscillator (APO)

In [30]:
def APO(symbol, numPeriodsFast, numPeriodsSlow):
    """
        Input:
            - symbol: the symbol of the stock for which the APO will be calculated to -> str
            - numPeriodsFast: number of time periods for the fast EMA -> int
            - numPeriodsSlow: number of time periods for the slow EMA -> int
        Output:
            APO values
    """
    apoValues = []

    emaFast = 0
    Kfast = 2 / (numPeriodsFast + 1) # fast EMA smoothing factor
    emaSlow = 0
    Kslow = 2 / (numPeriodsSlow + 1) # slow EMA smoothing factor
    
    for closePrice in indicators[symbol]['close']:
        if(emaFast == 0): # First observation
            emaFast = closePrice
            emaSlow = closePrice
        else:
            emaFast = (closePrice - emaFast) * Kfast + emaFast
            emaSlow = (closePrice - emaSlow) * Kslow + emaSlow
        apoValues.append(emaFast - emaSlow)
        
    return apoValues

### Moving Average Convergence Divergence (MACD)

In [31]:
def MACD(symbol, numPeriodsFast, numPeriodsSlow, numPeriodsMacd):
    """
        Input:
            - symbol: the symbol of the stock for which the APO will be calculated to -> str
            - numPeriodsFast: number of time periods for the fast EMA -> int
            - numPeriodsSlow: number of time periods for the slow EMA -> int
            - numPeriodsMacd: number of time periods for the MACD EMA -> int
        Output:
            MACD values
    """
    macdSignalValues = []
    macdHistogramValues = []

    emaFast = 0
    Kfast = 2 / (numPeriodsFast + 1) # fast EMA smoothing factor
    emaSlow = 0
    Kslow = 2 / (numPeriodsSlow + 1) # slow EMA smoothing factor
    emaMacd = 0
    Kmacd = 2 / (numPeriodsMacd + 1) # MACD EMA smoothing factor
    
    for closePrice in indicators[symbol]['close']:
        if(emaFast == 0): # First observation
            emaFast = closePrice
            emaSlow = closePrice
        else:
            emaFast = (closePrice - emaFast) * Kfast + emaFast
            emaSlow = (closePrice - emaSlow) * Kslow + emaSlow
        
        macd = emFast - emaSlow
        if(emaMacd == 0):
            emaMacd = macd
        else:
            emaMacd = (macd - emaMacd) * Kmacd + emaMacd
        
        macdSignalValues.append(emaMacd)
        macdHistogramValues.append(macd - emaMacd)
        
    return macdSignalValues, macdHistogramValues